## Лабораторная работа №1

**Тема:** word2vec и TF-IDF

**Выполнил:** Студент группы БВТ2201 Шамсутдинов Рустам Фаргатевич

**Цель лабораторной работы:** Изучить и реализовать алгоритмы word2vec и TF-IDF.

In [ ]:
import math
import re
from collections import Counter, defaultdict
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.datasets import fetch_20newsgroups

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

Загрузка и подготовка корпуса

In [ ]:
STOPWORDS = set(stopwords.words("english"))
LEMMA = WordNetLemmatizer()


def clean_text(text: str) -> str:
    # 1) Удаляем HTML-теги
    text = re.sub(r"<[^>]+>", " ", text)
    # 2) Удаляем URL
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    # 3) Ловим только английские буквы (удаляем цифры, пунктуацию)
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    # 4) Сводим к нижнему регистру
    text = text.lower()
    # 5) Нормализуем пробелы
    text = re.sub(r"\s+", " ", text).strip()
    return text


def tokenize(text: str) -> list[str]:
    cleaned = clean_text(text)
    tokens = cleaned.split()
    result = []
    for tok in tokens:
        # убираем короткие токены и стоп-слова
        if len(tok) < 3 or tok in STOPWORDS:
            continue
        # лемматизируем
        lemma = LEMMA.lemmatize(tok)
        result.append(lemma)
    return result


# Применение к DataFrame
newsgroups = fetch_20newsgroups(subset="train", remove=("headers", "footers", "quotes"))
docs = newsgroups.data[:200]

sentences = [tokenize(d) for d in docs]


df = pd.DataFrame({"text": docs})
df["tokens"] = df["text"].apply(tokenize)

# Посмотрим примеры
for i in range(3):
    print(df.loc[i, "tokens"][:10])

['wondering', 'anyone', 'could', 'enlighten', 'car', 'saw', 'day', 'door', 'sport', 'car']
['fair', 'number', 'brave', 'soul', 'upgraded', 'clock', 'oscillator', 'shared', 'experience', 'poll']
['well', 'folk', 'mac', 'plus', 'finally', 'gave', 'ghost', 'weekend', 'starting', 'life']


TF-IDF

In [ ]:
def compute_tf(tokens):
    counts = Counter(tokens)
    total = sum(counts.values())
    return {w: c / total for w, c in counts.items()}


df["tf"] = df["tokens"].apply(compute_tf)

Посчитать IDF по всему корпусу

In [ ]:
N = len(df)
df_unique = df["tokens"].apply(set)
df_counts = defaultdict(int)
for uniq in df_unique:
    for w in uniq:
        df_counts[w] += 1

idf = {w: math.log(N / df_counts[w]) for w in df_counts}

Собрать TF-IDF вектора (как словарь)

In [ ]:
def compute_tfidf(tf_dict):
    return {w: tf_dict[w] * idf[w] for w in tf_dict}


df["tfidf"] = df["tf"].apply(compute_tfidf)

# Пример: распечатаем топ-5 слов по tf-idf в первом документе
first_tfidf = df.loc[0, "tfidf"]
top5 = sorted(first_tfidf.items(), key=lambda x: x[1], reverse=True)[:5]
print("TF-IDF: топ-5 терминов первого документа:")
for term, value in top5:
    print(f"Слово: {term}, его значение: {value}")

TF-IDF: топ-5 терминов первого документа:
Слово: car, его значение: 0.2698067063953178
Слово: door, его значение: 0.24643336588595519
Слово: enlighten, его значение: 0.12321668294297759
Слово: bricklin, его значение: 0.12321668294297759
Слово: tellme, его значение: 0.12321668294297759


Простая Word2Vec (skip-gram без негативного сэмплинга)

In [ ]:
class FastWord2Vec:
    def __init__(self, sentences, window=2, dim=50, lr=0.025, epochs=3, neg_samples=5):
        # Строим словарь
        vocab = sorted({w for sent in sentences for w in sent})
        self.w2i = {w: i for i, w in enumerate(vocab)}
        self.i2w = {i: w for w, i in self.w2i.items()}
        self.V = len(vocab)
        self.dim = dim
        self.window = window
        self.lr = lr
        self.epochs = epochs
        self.neg = neg_samples

        # Инициализация весов
        self.W_in = np.random.randn(self.V, dim) * 0.01
        self.W_out = np.zeros((dim, self.V))

        # Распределение для negative sampling
        freq = Counter([w for sent in sentences for w in sent])
        pow_freq = np.array([freq[self.i2w[i]] ** 0.75 for i in range(self.V)])
        self.neg_dist = pow_freq / pow_freq.sum()

        # Корпус в индексах
        self.data = [[self.w2i[w] for w in sent] for sent in sentences]

    def train(self):
        for ep in range(1, self.epochs + 1):
            total_loss = 0.0
            for sent in self.data:
                for idx, target in enumerate(sent):
                    # контекстное окно
                    start = max(idx - self.window, 0)
                    end = min(idx + self.window + 1, len(sent))
                    contexts = [sent[i] for i in range(start, end) if i != idx]

                    v_in = self.W_in[target]
                    for ctx in contexts:
                        # негативные образцы
                        negs = list(
                            np.random.choice(self.V, size=self.neg, p=self.neg_dist)
                        )
                        samples = [ctx] + negs
                        labels = np.array([1] + [0] * self.neg)

                        vecs = self.W_out[:, samples]  # (dim, neg+1)
                        dots = v_in.dot(vecs)  # (neg+1,)
                        probs = 1 / (1 + np.exp(-dots))  # sigmoid

                        error = probs - labels
                        total_loss += -np.sum(
                            labels * np.log(probs + 1e-8)
                            + (1 - labels) * np.log(1 - probs + 1e-8)
                        )

                        # обновляем W_out и W_in
                        self.W_out[:, samples] -= self.lr * np.outer(v_in, error)
                        self.W_in[target] -= self.lr * (vecs.dot(error))
            print(f"Epoch {ep}/{self.epochs}, loss={total_loss:.4f}")

    def vector(self, word):
        idx = self.w2i.get(word)
        if idx is None:
            raise ValueError(f"'{word}' отсутствует в словаре")
        return self.W_in[idx]

    def most_similar(self, word, topn=5):
        if word not in self.w2i:
            raise ValueError(f"Слово '{word}' отсутствует в словаре.")
        v = self.vector(word)
        sims = np.dot(self.W_in, v) / (
            np.linalg.norm(self.W_in, axis=1) * np.linalg.norm(v) + 1e-9
        )
        best = np.argsort(-sims)
        result = []
        for idx in best:
            w = self.i2w[idx]
            if w != word:
                result.append((w, sims[idx]))
            if len(result) >= topn:
                break
        return result

    def most_similar_vector(self, vector, topn=5):
        sims = np.dot(self.W_in, vector) / (
            np.linalg.norm(self.W_in, axis=1) * np.linalg.norm(vector) + 1e-9
        )
        best = np.argsort(-sims)
        result = []
        for idx in best:
            w = self.i2w[idx]
            result.append((w, sims[idx]))
            if len(result) >= topn:
                break
        return result


Демонстрация работы

In [ ]:
model = FastWord2Vec(sentences, window=2, dim=50, lr=0.05, epochs=5, neg_samples=5)
model.train()


Epoch 1/5, loss=291331.6168
Epoch 2/5, loss=267609.1142
Epoch 3/5, loss=220984.4192
Epoch 4/5, loss=196768.6364
Epoch 5/5, loss=185811.3838


In [ ]:
# w1, w2 = random.sample(list(model.w2i.keys()), 2)
w1 = "king"
w2 = "woman"
v1, v2 = model.vector(w1), model.vector(w2)

diff = v1 - v2
summ = v1 + v2

print(f"\nСлова: '{w1}' vs '{w2}'")
print("Разность (первые 5):", diff[:5])
print("Сумма   (первые 5):", summ[:5])


Слова: 'king' vs 'woman'
Разность (первые 5): [-0.19782094  0.05931985  0.10631466  0.12400341 -0.11832959]
Сумма   (первые 5): [ 0.61607477  0.23506817  0.47135306 -0.84110859  0.83965184]


In [21]:
similar_words_diff = model.most_similar_vector(diff, topn=5)
print(f"Топ-5 слов, близких к вектору разности '{w1}' - '{w2}':")
for word, score in similar_words_diff:
    print(f"{word}: {score:.4f}")

Топ-5 слов, близких к вектору разности 'king' - 'woman':
rlk: 0.4465
ltq: 0.4135
pmf: 0.3699
whjn: 0.3611
fyn: 0.3600


In [22]:
similar_words_diff = model.most_similar_vector(summ, topn=5)
print(f"Топ-5 слов, близких к вектору суммы '{w1}' + '{w2}':")
for word, score in similar_words_diff:
    print(f"{word}: {score:.4f}")

Топ-5 слов, близких к вектору суммы 'king' + 'woman':
swear: 0.9941
turned: 0.9929
story: 0.9914
home: 0.9910
whether: 0.9910


In [25]:
w1 = "king"
w2 = "woman"
w3 = "man"
v1, v2, v3 = model.vector(w1), model.vector(w2), model.vector(w3)
test = v1 - v3 + v2

similar_words_test = model.most_similar_vector(test, topn=5)
print(f"Тeстирование нескольких операций")
for word, score in similar_words_test:
    print(f"{word}: {score:.4f}")

Тeстирование нескольких операций
woman: 0.9876
never: 0.9665
child: 0.9654
kurdish: 0.9631
hundred: 0.9629
